In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'utilizando dispositivo {device}')

utilizando dispositivo cuda


In [3]:
# transformação para o pre processamento das imagens
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(), # converter pra tensor
    transforms.Normalize((0.5,),(0.5,)) #normalização para [-1, 1]
])

In [4]:
dataset = datasets.ImageFolder(root='PetImages', transform=transform)

In [5]:
# definir proporções entre treino e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [6]:
# dividir dados aleatoriamente
train_data, test_data = random_split(dataset, [train_size, test_size])

In [7]:
# criar data loader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [8]:
print(f'tamanho do conjunto de treino: {len(train_data)} imagens')
print(f'tamanho do conjunto de teste: {len(test_data)} imagens')

tamanho do conjunto de treino: 19998 imagens
tamanho do conjunto de teste: 5000 imagens


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2) # saida de duas classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16) # flatten
        x = self.relu(self.fc1(x)) 
        x = self.fc2(x) #Removemos Softmax porque CrossEntropyLoss já lida com isso
        return x

In [10]:
# criar a rede e mover pra GPU
modelo = CNN().to(device)

# função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr = 0.001)

In [11]:
# treino
def train(modelo, train_loader, criterion, optimizer, epochs = 10):
    modelo.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            print(f"Época {epoch + 1} Loss: {running_loss/len(train_loader):.4f}")


In [12]:
# teste
def test(modelo, test_loader):
    modelo.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelo(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Acurácia: {100 * correct / total:.2f}%")

In [13]:
# treinamento
train(modelo, train_loader, criterion, optimizer, epochs = 50)

Época 1 Loss: 0.0011
Época 1 Loss: 0.0053
Época 1 Loss: 0.0065
Época 1 Loss: 0.0079
Época 1 Loss: 0.0092
Época 1 Loss: 0.0104
Época 1 Loss: 0.0115
Época 1 Loss: 0.0126
Época 1 Loss: 0.0137
Época 1 Loss: 0.0148
Época 1 Loss: 0.0160
Época 1 Loss: 0.0171
Época 1 Loss: 0.0181
Época 1 Loss: 0.0192
Época 1 Loss: 0.0203
Época 1 Loss: 0.0215
Época 1 Loss: 0.0226
Época 1 Loss: 0.0237
Época 1 Loss: 0.0248
Época 1 Loss: 0.0259
Época 1 Loss: 0.0270
Época 1 Loss: 0.0281
Época 1 Loss: 0.0292
Época 1 Loss: 0.0303
Época 1 Loss: 0.0314
Época 1 Loss: 0.0325
Época 1 Loss: 0.0336
Época 1 Loss: 0.0347
Época 1 Loss: 0.0358
Época 1 Loss: 0.0369
Época 1 Loss: 0.0381
Época 1 Loss: 0.0392
Época 1 Loss: 0.0403
Época 1 Loss: 0.0414
Época 1 Loss: 0.0425
Época 1 Loss: 0.0435
Época 1 Loss: 0.0447
Época 1 Loss: 0.0458
Época 1 Loss: 0.0470
Época 1 Loss: 0.0480
Época 1 Loss: 0.0491
Época 1 Loss: 0.0502
Época 1 Loss: 0.0513
Época 1 Loss: 0.0524
Época 1 Loss: 0.0535
Época 1 Loss: 0.0546
Época 1 Loss: 0.0557
Época 1 Loss:

C:\Users\Mateus\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Época 1 Loss: 0.3328
Época 1 Loss: 0.3338
Época 1 Loss: 0.3347
Época 1 Loss: 0.3354
Época 1 Loss: 0.3364
Época 1 Loss: 0.3371
Época 1 Loss: 0.3379
Época 1 Loss: 0.3386
Época 1 Loss: 0.3395
Época 1 Loss: 0.3404
Época 1 Loss: 0.3412
Época 1 Loss: 0.3418
Época 1 Loss: 0.3428
Época 1 Loss: 0.3437
Época 1 Loss: 0.3445
Época 1 Loss: 0.3455
Época 1 Loss: 0.3463
Época 1 Loss: 0.3470
Época 1 Loss: 0.3477
Época 1 Loss: 0.3484
Época 1 Loss: 0.3496
Época 1 Loss: 0.3503
Época 1 Loss: 0.3512
Época 1 Loss: 0.3523
Época 1 Loss: 0.3532
Época 1 Loss: 0.3541
Época 1 Loss: 0.3549
Época 1 Loss: 0.3560
Época 1 Loss: 0.3571
Época 1 Loss: 0.3581
Época 1 Loss: 0.3589
Época 1 Loss: 0.3600
Época 1 Loss: 0.3609
Época 1 Loss: 0.3617
Época 1 Loss: 0.3628
Época 1 Loss: 0.3637
Época 1 Loss: 0.3645
Época 1 Loss: 0.3653
Época 1 Loss: 0.3663
Época 1 Loss: 0.3672
Época 1 Loss: 0.3682
Época 1 Loss: 0.3690
Época 1 Loss: 0.3699
Época 1 Loss: 0.3706
Época 1 Loss: 0.3715
Época 1 Loss: 0.3723
Época 1 Loss: 0.3732
Época 1 Loss:

In [14]:
# teste
test(modelo, test_loader)

Acurácia: 78.58%


In [15]:
torch.save(modelo.state_dict(), "modelo_cachorros_gatos.pth")
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
